<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisCuantitativoAvanzado/blob/main/SeriesdeTiempo/EMAE_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json

import requests
from requests.auth import HTTPBasicAuth

import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
from google.colab import userdata

# Definir user key para la API
user_key = userdata.get('ALPHACAST')

In [63]:
# EMAE
emae = requests.get('https://api.alphacast.io/datasets/5331/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('emae.csv','wb').write(emae.content)
emae = pd.read_csv('emae.csv',delimiter = ',')

In [67]:
emae.set_index('Date', inplace = True)
emae = emae.loc['2016-02-01':]
y=emae['Emae - sa_orig - current_prices_mom']

In [100]:
y

,Emae - sa_orig - current_prices_mom
Date,
2016-02-01,-0.732842
2016-03-01,-0.542077
2016-04-01,-0.808724
2016-05-01,-0.334856
2016-06-01,-0.207642
...,...
2024-03-01,-1.365632
2024-04-01,-1.041793
2024-05-01,0.544140


In [74]:
# IPI
ipi = requests.get('https://api.alphacast.io/datasets/5679/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('ipi.csv','wb').write(ipi.content)
ipi = pd.read_csv('ipi.csv',delimiter = ',')


ipi['Date'] = pd.to_datetime(ipi['Date'])
ipi.set_index('Date', inplace = True)

In [75]:
ipi.head()

,Country,Division,Seasonal Adjustment,Subclass,Value,Value - current_prices_3m_yoy,Value - current_prices_yoy,Value - current_prices_mom
Date,,,,,,,,
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,"Azúcar, productos de confitería y chocolate",50.680600,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Carne aviar,212.009238,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Carne vacuna,83.169668,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,Fiambres y embutidos,143.350871,NaN,NaN,NaN
2016-01-01,Argentina,Alimentos y bebidas,Not adjusted,"Galletitas, productos de panadería y pastas",95.801964,NaN,NaN,NaN


In [76]:
ipi[(ipi['Seasonal Adjustment']== "Seasonally Adjusted")].head()

,Country,Division,Seasonal Adjustment,Subclass,Value,Value - current_prices_3m_yoy,Value - current_prices_yoy,Value - current_prices_mom
Date,,,,,,,,
2016-01-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,132.565049,NaN,NaN,NaN
2016-02-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,135.260212,NaN,NaN,2.033088
2016-03-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,132.734580,NaN,NaN,-1.867240
2016-04-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,129.439709,NaN,NaN,-2.482300
2016-05-01,Argentina,IPI Manufacturero,Seasonally Adjusted,none,128.673810,NaN,NaN,-0.591704


In [77]:
ipi = ipi[(ipi['Division'] == "IPI Manufacturero") & (ipi['Seasonal Adjustment'] == "Seasonally Adjusted")]
ipi = ipi['Value - current_prices_mom']
ipi.columns = ['Value - sa_MoM']
ipi = ipi.loc['2016-02-01':]

In [83]:
# ISAC
isac = requests.get('https://api.alphacast.io/datasets/5565/data?apiKey='+user_key+'&&$format=csv', auth=HTTPBasicAuth(user_key, ''))
open('isac.csv','wb').write(isac.content)
isac = pd.read_csv('isac.csv',delimiter = ',')

In [86]:
isac['Date'] = pd.to_datetime(isac['Date'])
isac.set_index('Date', inplace = True)
isac = isac['Nivel general - sa_orig - current_prices_mom']
isac.columns = ['Nivel general - sa_MoM']
isac = isac.loc['2016-02-01':]

In [87]:
isac=pd.Series(isac, name='ISAC')
ipi=pd.Series(ipi, name='IPI')

In [88]:
X = pd.concat([ipi, isac], axis=1)

In [89]:
X

,IPI,ISAC
Date,,
2016-02-01,2.033088,-1.718933
2016-03-01,-1.867240,-8.620907
2016-04-01,-2.482300,-5.251907
2016-05-01,-0.591704,5.275489
2016-06-01,-1.918772,-4.194349
...,...,...
2024-03-01,-3.126307,-15.388856
2024-04-01,0.069197,1.122882
2024-05-01,-0.132622,7.730018


In [90]:
y.shape

(102,)

In [91]:
X.shape

(102, 2)

In [92]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X[:-1], y[:-1]) #Si ya salió IPI e ISAC, pero todavía no el EMAE entonces hay que fitear lr.fit(X[:-1], y) si ya salió todo, lr.fit(X[:-1], y[:-1])
Yest = lr.predict(X[:-1])

In [93]:
Yest

array([ 0.42702653, -1.19072616, -1.12724862,  0.17023366, -0.88919887,
       -0.07360048,  0.23552553,  0.76836925, -0.41732174,  0.65882151,
        0.08488566,  0.02069667, -1.7627185 ,  1.67146637,  0.27965844,
        0.33645236,  1.07106712,  0.32556039, -0.13335625,  0.15142309,
       -0.04305912,  0.96538613, -0.67989889, -0.88295521, -1.17096713,
        1.77499745, -0.46586477, -0.87792618, -1.13603737, -0.61087092,
        0.32172625, -0.76242108, -0.6287656 , -1.1727317 , -0.18278369,
        0.55676007,  0.44124718, -0.8051074 ,  1.1468121 , -0.18255074,
       -0.52631849,  0.05783761, -0.22950389, -0.73647386,  0.22992131,
       -1.08373801, -0.07983438,  0.05595491,  0.18992336, -8.4290211 ,
       -8.13453814, 13.03850821,  8.10130546,  0.65583684,  0.75280911,
        0.89443554,  0.34801251,  2.53896342, -0.28578926,  1.60192132,
       -1.37162736,  0.98483883, -1.2503963 , -0.49246324,  1.60235647,
       -0.21096695, -0.08963437,  0.31314111, -0.59688273,  1.88

In [94]:
print('DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE')
print()
print('Valor de la pendiente o coeficiente "a":')
print(lr.coef_)
print('Valor de la intersección o coeficiente "b":')
print(lr.intercept_)
print()
print('La ecuación del modelo es igual a:')
print('y = ', lr.coef_, 'X + ', lr.intercept_)
print()
print('Precisión del modelo:')
print(lr.score(X[:-1], y[:-1]))

DATOS DEL MODELO REGRESIÓN LINEAL SIMPLE

Valor de la pendiente o coeficiente "a":
[0.28829407 0.07147395]
Valor de la intersección o coeficiente "b":
-0.03624168925378102

La ecuación del modelo es igual a:
y =  [0.28829407 0.07147395] X +  -0.03624168925378102

Precisión del modelo:
0.734039952702229


In [95]:
lr.coef_[0]

0.2882940661181749

In [97]:
X[-1:]

,IPI,ISAC
Date,,
2024-07-01,6.873029,7.969635


In [98]:
emae_predict=X[-1:]['IPI']*lr.coef_[0]+X[-1:]['ISAC']*lr.coef_[1]+lr.intercept_

In [99]:
emae_predict

,0
Date,
2024-07-01,2.514833
